#### Libraries importeren 

In [235]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

#### Random seed zetten

In [236]:
torch.manual_seed(42)
np.random.seed(42)

#### Dataset inladen

In [237]:
kolommen = ["animal_name", "hair", "feathers", "eggs", "milk", "airborne",
           "aquatic", "predator", "toothed", "backbone", "breathes",
           "venomous", "fins", "legs", "tail", "domestic", "catsize", "type"]

dieren_df = pd.read_csv("Data/zoo.data", names = kolommen)
dieren_df

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,worm,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


#### Train & Test data maken

In [238]:
dieren_df = dieren_df.drop(columns=["animal_name"])
x = dieren_df.drop(columns=["type"])
y = dieren_df["type"]

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state= 42)

# Omzetten naar tensors voor training en testen
# -1 omdat type ipv 1-7 naar 0-6 moet (anders is hij out of bounds) 
X_train = torch.tensor(X_train.values, dtype = torch.float32)
Y_train = torch.tensor(Y_train.values, dtype = torch.long) - 1

X_test = torch.tensor(X_test.values, dtype = torch.float32)
Y_test = torch.tensor(Y_test.values, dtype = torch.long) - 1

#### Data inspecteren

In [239]:
print(X_train.shape)

torch.Size([80, 16])


In [240]:
print(Y_train.shape)

torch.Size([80])


#### Simpel neuraal netwerk bouwen

In [241]:
model = nn.Sequential(
    nn.Linear(16, 10),
    nn.ReLU(),
    nn.Linear(10, 7)
)

#### Loss uitrekenen en training loop starten

In [242]:
criterium = nn.CrossEntropyLoss()
optimalisator = optim.Adam(model.parameters(), lr = 0.01)

epochs = 100
for epoch in range(epochs):
    model.train()
    optimalisator.zero_grad()
    outputs = model(X_train)
    loss = criterium(outputs, Y_train)
    loss.backward()
    optimalisator.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, loss: {loss.item():.4f}")

print("Training voltooid!")

Epoch 0/100, loss: 1.9355
Epoch 10/100, loss: 1.5271
Epoch 20/100, loss: 1.1084
Epoch 30/100, loss: 0.7817
Epoch 40/100, loss: 0.5602
Epoch 50/100, loss: 0.4124
Epoch 60/100, loss: 0.3094
Epoch 70/100, loss: 0.2343
Epoch 80/100, loss: 0.1795
Epoch 90/100, loss: 0.1385
Training voltooid!


#### Uitkomsten testen

In [243]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
        
    _, predicted = torch.max(test_outputs, 1)
    correct_voorspeld = (predicted == Y_test).sum().item()
    totaal_voorspeld = Y_test.size(0)
    accuratie = correct_voorspeld / totaal_voorspeld * 100
    
    print("Werkelijke labels:", Y_test[:20].numpy())  
    print("Voorspelde labels:", predicted[:20].numpy()) 
    print(f"Accuratie score: {accuratie:.2f}")

resultaten = pd.DataFrame({
    "Werkelijke tabel": Y_test.numpy().flatten(),
    'Voorspelde label': predicted.numpy().flatten()
})

Werkelijke labels: [0 0 0 0 0 5 0 0 0 0 3 5 5 1 6 0 0 1 3 0]
Voorspelde labels: [0 0 0 0 0 5 0 0 0 0 3 5 5 1 6 0 0 1 3 0]
Accuratie score: 95.24


In [ ]:
resultaten

42